In [ ]:
# Import packages
import pandas as pd
import sqlalchemy as sa

# Create a SQLite file named data.db
engine = sa.create_engine("sqlite:///data.db")

# Read the four practice data points files
train1 = pd.read_csv("train1.csv")
# don't have the files
# train2 = pd.read_csv("train2.csv")
# train3 = pd.read_csv("train3.csv")
# train4 = pd.read_csv("train4.csv")

# Combine them into one dataframe with four columns for y
train = pd.DataFrame()
train["X"] = train1["x"]
train["Y1"] = train1["y1"]
# don't have the files
# train["Y2"] = train2["y"]
# train["Y3"] = train3["y"]
# train["Y4"] = train4["y"]

# Save the dataframe as a table named train_table in the SQLite file with if_exists='replace' to avoid table already exists error
train.to_sql("train_table", engine, index=False, if_exists='replace')

# Read the 50 curves files
ideal = pd.DataFrame()
ideal["X"] = pd.read_csv("ideal1.csv")["x"]
ideal["Y1"] = pd.read_csv("ideal1.csv")["y1"]

# Save the dataframe as a table named ideal_table in the SQLite file with if_exists='replace' to avoid table already exists error
ideal.to_sql("ideal_table", engine, index=False, if_exists='replace')

400

In [ ]:
# Import packages
import sqlalchemy as sa
import numpy as np

# Connect to the SQLite file
engine = sa.create_engine("sqlite:///data.db")

# Query the train_table and get the data points as a numpy array
train_data = np.array(engine.execute("SELECT * FROM train_table").fetchall())

# Query the ideal_table and get the curves as a numpy array
ideal_data = np.array(engine.execute("SELECT * FROM ideal_table").fetchall())

# Create an empty dictionary to store the results
results = {}

# Loop through each practice data point
for i in range(len(train_data)):
    # Get the x and y values of the data point
    x = train_data[i, 0]
    y = train_data[i, 1]
    # Create an empty list to store the least-square values for each curve
    lsq_list = []
    # Loop through each curve
    for j in range(len(ideal_data)):
        # Get the x and y values of the curve
        x_curve = ideal_data[j, 0]
        y_curve = ideal_data[j, 1]
        # Check if the x values match
        if x == x_curve:
            # Calculate the difference between the y values
            diff = y - y_curve
            # Square the difference and append it to the list
            lsq_list.append(diff ** 2)
    # Find the minimum least-square value and its index
    min_lsq = min(lsq_list)
    min_index = lsq_list.index(min_lsq)
    # Add the number of the curve and the least-square value to the results dictionary
    results[i] = (min_index + 1, min_lsq)


In [ ]:
import pprint
pprint.pprint(results)

{0: (1, 1969.5306739383886),
 1: (1, 1892.0164905043932),
 2: (1, 1914.2632538886749),
 3: (1, 1936.9572555795573),
 4: (1, 1892.8359381521434),
 5: (1, 1911.415940227577),
 6: (1, 1912.7016138563583),
 7: (1, 1822.4104562072102),
 8: (1, 1831.7685315450915),
 9: (1, 1853.5391792359017),
 10: (1, 1825.4722639078975),
 11: (1, 1839.7972084733776),
 12: (1, 1796.8908745418805),
 13: (1, 1786.0251709453364),
 14: (1, 1783.976557978349),
 15: (1, 1773.7116871534108),
 16: (1, 1743.649189464477),
 17: (1, 1744.7023746175175),
 18: (1, 1735.510840784584),
 19: (1, 1770.339600725514),
 20: (1, 1771.3944602764202),
 21: (1, 1724.8264644895883),
 22: (1, 1708.341842322733),
 23: (1, 1688.1536042254904),
 24: (1, 1710.521179610256),
 25: (1, 1642.0786794049002),
 26: (1, 1630.2778544379603),
 27: (1, 1687.4950415079616),
 28: (1, 1592.1508580955779),
 29: (1, 1580.0563785059292),
 30: (1, 1575.364166703855),
 31: (1, 1573.7028701657762),
 32: (1, 1556.5227208459044),
 33: (1, 1530.0313172704175)

In [ ]:
# Import packages
import pandas as pd
import sqlalchemy as sa
import numpy as np

# Connect to the SQLite file
engine = sa.create_engine("sqlite:///data.db")

# Read the test.csv file and get the test data points as a dataframe
test = pd.read_csv("test.csv")

# Create an empty list to store the numbers of the four curves we picked
curve_list = []

# Loop through the results dictionary
for key, value in results.items():
    # Get the number of the curve that fits each practice data point
    curve_num = value[0]
    # Append it to the list if it is not already there
    if curve_num not in curve_list:
        curve_list.append(curve_num)

# Sort the list in ascending order
curve_list.sort()

# Create an empty numpy array to store the four curves we picked
curves = np.empty((len(curve_list), 400))

# Loop through the list of curve numbers
for i in range(len(curve_list)):
    # Query the ideal_table and get the x and y values of each curve as a numpy array
    curve_data = np.array(engine.execute(f"SELECT X, Y{curve_list[i]} FROM ideal_table").fetchall())
    # Assign it to the corresponding row of the curves array
    curves[i] = curve_data[:, 1]


In [ ]:
# Create an empty dictionary to store the test results
test_results = {}

# Loop through each test data point
for i in range(len(test)):
    # Get the x and y values of the test data point
    x_test = test.iloc[i, 0]
    y_test = test.iloc[i, 1]
    # Create an empty list to store the difference values for each curve
    diff_list = []
    # Loop through each curve
    for j in range(len(curves)):
        # Get the y value of the curve at the same x value as the test data point
        y_curve = curves[j, int(x_test) + 20]
        # Calculate the difference between the y values
        diff = abs(y_test - y_curve)
        # Append it to the list
        diff_list.append(diff)
    # Find the minimum difference value and its index
    min_diff = min(diff_list)
    min_index = diff_list.index(min_diff)
    # Get the number of the curve that has the minimum difference
    curve_num = curve_list[min_index]
    # Get the biggest difference we found when fitting the practice data points with this curve
    max_diff = results[min_index][1]
    # Check if the minimum difference is not more than sqrt(2) times bigger than the biggest difference
    if min_diff <= np.sqrt(2) * max_diff:
        # If yes, then it fits this curve
        fit = curve_num
    else:
        # If no, then it doesn't fit any curve
        fit = None
    # Add the number of the curve that fits this test data point, or None if none fits, to the test results dictionary
    test_results[i] = (fit, min_diff)


In [ ]:
pprint.pprint(test_results)

{0: (None, 4494.544634649999),
 1: (1, 78.04443754),
 2: (1, 1.5645722000000002),
 3: (None, 7730.90445714),
 4: (None, 13841.22206575),
 5: (1, 0.3982425),
 6: (1, 57.42786776),
 7: (1, 1317.97763564),
 8: (1, 22.1124519),
 9: (1, 34.44866425),
 10: (1, 2.10223123),
 11: (None, 14732.68848063),
 12: (1, 48.757787539999995),
 13: (None, 7739.53706014),
 14: (1, 33.234994650000004),
 15: (1, 11.235161000000002),
 16: (1, 105.88952277),
 17: (None, 4194.42228508),
 18: (1, 50.7916012),
 19: (1, 11.6051796),
 20: (1, 57.04199576),
 21: (None, 5849.348071400001),
 22: (1, 5.5258948299999995),
 23: (1, 1363.3522999000002),
 24: (1, 8.93624554),
 25: (1, 14.603385000000001),
 26: (1, 1.54660926),
 27: (1, 11.58880684),
 28: (None, 10422.70136535),
 29: (1, 45.7028858),
 30: (1, 2589.55890253),
 31: (1, 44.878121),
 32: (1, 76.7666947),
 33: (1, 151.3838272),
 34: (1, 1.4847867700000001),
 35: (None, 4671.1161672),
 36: (1, 69.03044286),
 37: (1, 199.67406437),
 38: (1, 43.359439900000005),
 

In [ ]:
# Create a dataframe from the test results dictionary
test_results_df = pd.DataFrame.from_dict(test_results, orient="index", columns=["N", "D"])

# Add the x and y columns from the test dataframe
test_results_df["X"] = test["x"]
test_results_df["Y"] = test["y"]

# Reorder the columns
test_results_df = test_results_df[["X", "Y", "N", "D"]]

# Save the dataframe as a table named test_table in the SQLite file with if_exists='replace' to avoid table already exists error
test_results_df.to_sql("test_table", engine, index=False, if_exists='replace')

100